# *신경망 학습*

## 2. 손실함수

> - 신경망 성능의 '나쁨'을 나타내는 지표
- 일반적으로 평균제곱 오차와 교차 엔트로피 오차를 사용
- '나쁨'을 최소화 하는 가중치 매개변수 값을 탐색

___

### 1) 평균 제곱 오차

- 가장 많이 쓰이는 손실 함수로 **MSE, Mean Squared Error(평균제곱오차)** 라고 함
- 수식으로는 아래와 같으며
$$
E = \frac{1}{2}\sum_k(y_k - t_k)^2
$$
- $y_k$는 신경망의 예측값, $t_k$는 정답 레이블, $k$는 데이터의 차원수를 의미
- 예를 들면 '3.6' 손글씨 숫자 인식에서의 $y_k$와 $t_k$는 다음과 같음

In [3]:
y = [.1, .05, .6, 0, .05, .1, 0, .1, 0 , 0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
print(y)
print(t)

[0.1, 0.05, 0.6, 0, 0.05, 0.1, 0, 0.1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


- 위 예에서 t 객체의 2번 인덱스가 정답을 가르키는 위치로 1, 그외는 0 을 의미
- y의 소프트맥스 출렵값도 2번 인덱스가 0.6으로 제일 높으며 출력 확률은 0.6으로 해석가능
- *t 처럼 한 원소만 1로하고 그 외는 0으로 나타내는 표기법을 '원-핫 인코딩'이라고 함*

- 평균제곱오차는 아래와 같이 구할 수 있음

In [14]:
def mean_squared_error (y, t):
    return 0.5 * np.sum((np.array(y)-np.array(t))**2)

In [15]:
mean_squared_error(x,y)

0.09750000000000003

In [16]:
# y에서 7번째 인덱스를 정답으로 추정했다면...

y = [.1, .05, .1, 0, .05, .1, 0, .6, 0 , 0]
mean_squared_error(y,t )


0.5975

- 정답에 가깔울 때 손신함수 쪽 출력이 작으며, 당연히 오차도 더 작음
___

### 2) 교차 엔트로피 오차

- **CEE, Cross Entropy Error(교차 엔트로피 오차)**도 자주 이용하는 손실함수 중 하나
- 수식은 다음과 같으며
$$
E = -\sum_kt_klogy_k
$$
- 마찬가지로 $y_k$는 신경망의 예측값, $t_k$는 정답 레이블, $k$는 데이터의 차원수를 의미

- 여기서 $t_k$는 원-핫 인코딩을 쓰기 때문에 위의 식은 **정답일 경우에만 자연로그의 값을 구하는 식**이 됨

In [18]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(np.array(t) * np.log(np.array(y) + delta))

- delta는 log계산 안쪽 값이 0이 되어 -Inf 값을 반환하는 것을 방지

In [20]:
y = [.1, .05, .6, 0, .05, .1, 0, .1, 0 , 0]
cross_entropy_error(y, t)

0.510825457099338

In [21]:
y = [.1, .05, .1, 0, .05, .1, 0, .6, 0 , 0]
cross_entropy_error(y, t)

2.302584092994546

![](image/fig 4-3.png)

- 자연로그의 그래프를 보면 정답일 때 (x==1) y 값은 0이 되고, 에러가 높을 때 (x가 0에 수렴할 때) y 값은 점점 작아짐
- -자연로그를 취함으로 손실 값을 계산
___

### 3) 미니배치 학습
- 기계학습은 훈련 데이터에 대한 손실함수의 값을 구하고, 그 값을 최대한 줄여주는 매개변수를 찾음
- 따라서 전체 훈련 데이터를 대상으로 손실 함수의 값을 구하고, **손실함수 값들의 합을 지표로 설정**
- 교차 엔트로피의 경우 다음과 같이 수식으로 설정 가능
$$
E = -\frac{1}{N}\sum_n\sum_kt_{nk}logy_{nk}
$$

- MNIST의 데이터셋은 훈련 데이터가 60,000개, 빅데이터 수준이 되면 모든 데이터를 대상으로 손실함수를 계산하는 것은 비현실적
- 따라서 데이터 일부만 추려 '근사치'로 이용하는 **mini-batch(미니배치)**와 **미니배치 학습**을 이용

In [24]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from data.mnist import load_mnist

In [26]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


- 훈련 데이터는 60,000개에 입력 데이터가 784개(28*28)
- 여기서 데이터 10개만 무작위로 추출하기 위해서 np.random.choice() 함수 사용

In [29]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

___
### 4) (배치용) 교차 엔트로피 오차 구현하기

In [30]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(np.array(t) * np.log(np.array(y) + delta))

- 위 식을 수정하여 하나인 경우와 배치의 경우 모두를 처리할 수 있도록 구현

In [31]:
def cross_entropy_error(y,t):
    if y.ndim ==1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(np.array(t) * np.log(np.array(y) + 1e-7)) / batch_size

- 아래의 경우 원-핫 인코딩 대신 '2'나 '7' 같은 숫자 레이블이 값으로 주어진 경우임
- 다른점은 정답에 해당하는 신경망의 출력만으로 오차를 계산 (t가 0인 값 제외)

In [32]:
def cross_entropy_error(y,t):
    if y.ndim ==1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arrange(batc_size), t]) + 1e-7) / batch_size

___
### 5) 왜 손실함수를 설정하는가?
- '정확도'라는 명확한 지표 대신 '손실함수의 값'이라는 우회적인 방법을 택하는 이유는 **'미분'**에 있음
- 가중치 매개변수의 값을 아주 조금 변화 시켰을 때 손실함수가 어떻게 변하는지를 보고 어느 방향으로 매개변수를 갱신할 지 계산
- 반대로 정확도가 지표가 되었을 때 대부분의 경우 미분 값이 0이 되어 갱신이 불가능

- 예를 들어 정확도 지표를 사용 할 경우 매개변수를 조정한다 해도 그 값은 연속적이지 않고 갑자기 값이 변하게 됨 (32/100) > (33/100)
- 손실함수는 매개 변수 조정에 따라 값이 연속적으로 변화하며 어느 장소에서도 기울기가 0이 되지 않음(거의)